In [109]:
'''
Source: https://medium.datadriveninvestor.com/scraping-live-stock-fundamental-ratios-news-and-more-with-python-a716329e0493

'''

'\nSource: https://medium.datadriveninvestor.com/scraping-live-stock-fundamental-ratios-news-and-more-with-python-a716329e0493\n\n'

In [110]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen, Request

In [111]:
# Input symbol
symbol = 'TSLA'

# Set up scraper
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

In [112]:
# Locate fundamentals table
fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]

In [113]:
# Reshape dataframe to get two columns dataframe and change column names
columns = fundamentals.values.reshape(-1,2)
fundamentals = pd.DataFrame(columns)
fundamentals['Fundamentals'] = fundamentals[0]
fundamentals['Values'] = fundamentals[1]
fundamentals = fundamentals.drop(fundamentals.columns[[0,1]], axis=1)
fundamentals

,Fundamentals,Values
0,Index,S&P 500
1,P/E,1088.42
2,EPS (ttm),0.62
3,Insider Own,0.10%
4,Shs Outstand,959.85M
...,...,...
67,SMA20,-4.54%
68,SMA50,-1.89%
69,SMA200,20.13%
70,Volume,28711915


In [114]:
# Locate news table
news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]

In [115]:
# Change column names 
news['Date'] = news[0]
news['Title'] = news[1]
news = news.drop(news.columns[[0,1]], axis=1)
news.head()

,Date,Title
0,Apr-29-21 10:21PM,"Dow Jones Futures: Amazon, Apple Suppliers, Ni..."
1,08:36PM,Nio Earnings Mixed As Q2 Deliveries Seen Nearl...
2,05:10PM,Straddle Option Price Useful For Earnings Move...
3,04:40PM,ARK Invest Stocks To Buy And Watch: 6 Stocks T...
4,01:35PM,Tesla slips below 10% of Cathie Wood's flagshi...


In [116]:
# Locate href (news url) in news table and create a dataframe of  news links
news_link = html.find_all('a', class_ ='tab-link-news')
links = []
for a in news_link:
    links.append(a['href'])
links = pd.DataFrame(links)
links['Link'] = links
links = links.drop(links.columns[0], axis=1)
links.head()

,Link
0,https://www.investors.com/market-trend/stock-m...
1,https://www.investors.com/news/nio-stock-nio-e...
2,https://www.investors.com/research/options/str...
3,https://www.investors.com/research/ark-invest-...
4,https://finance.yahoo.com/news/tesla-slips-bel...


In [117]:
# Concat news titles and news links into one dataframe, set date as index
news = pd.concat([news, links], join='inner', axis=1)
news = news.set_index('Date')
news.head()

,Title,Link
Date,,
Apr-29-21 10:21PM,"Dow Jones Futures: Amazon, Apple Suppliers, Ni...",https://www.investors.com/market-trend/stock-m...
08:36PM,Nio Earnings Mixed As Q2 Deliveries Seen Nearl...,https://www.investors.com/news/nio-stock-nio-e...
05:10PM,Straddle Option Price Useful For Earnings Move...,https://www.investors.com/research/options/str...
04:40PM,ARK Invest Stocks To Buy And Watch: 6 Stocks T...,https://www.investors.com/research/ark-invest-...
01:35PM,Tesla slips below 10% of Cathie Wood's flagshi...,https://finance.yahoo.com/news/tesla-slips-bel...


In [118]:
# Locate insider trading table, and set the first row as column title
insider_trading = pd.read_html(str(html), attrs = {'class':'body-table'})[0]
insider_trading.columns = insider_trading.iloc[0]
insider_trading = insider_trading.drop([0], axis=0)
insider_trading.head()

,Insider Trading,Relationship,Date,Transaction,Cost,#Shares,Value ($),#Shares Total,SEC Form 4
1,Baglino Andrew D,SVP Powertrain and Energy Eng.,Apr 27,Option Exercise,41.57,1000,41570,20114,Apr 29 08:20 PM
2,Baglino Andrew D,SVP Powertrain and Energy Eng.,Apr 27,Sale,717.17,1500,1075755,18614,Apr 29 08:20 PM
3,DENHOLM ROBYN M,Director,Apr 26,Option Exercise,52.38,62500,3273750,67500,Apr 28 09:41 PM
4,DENHOLM ROBYN M,Director,Apr 26,Sale,740.97,62500,46310388,5000,Apr 28 09:41 PM
5,Kirkhorn Zachary,Chief Financial Officer,Apr 19,Sale,718.00,1250,897500,55984,Apr 21 08:23 PM
